In [1]:
import os
import pandas as pd
import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import itertools
import collections as coll
import csv
#import time
#from mlxtend.frequent_patterns import fpgrowth

## Data Cleaning

In [2]:
# attempt to get all the files within the balnton_data directory
path = r"./blanton_data/"
directories = os.listdir(path)

# this is the master dataframe
df_master = pd.DataFrame()

# these are the transaction databases, stored as indexes in a dictionary
# dictionary of dictionaries of arrays
transaction_database = {
    'emotional' : {},
    'aesthetic' : {},
    'either' : {}
}

valid_cols = ['accession_#', 'artist_sort_name', 'artist_life_dates',
           'artist_nationality', 'title', 'creation_date', 'medium', 'credit_line',
           'dimensions', 'student_id', 'emotional_reaction', 'aesthetically_pleasing']

In [3]:
def update_transaction_database(student_df):
    # get the student_id to create a new entry in the transaction_database
    student_id = student_df.at[0, 'student_id']
    
    # create new entry in database
    transaction_database['emotional'][student_id] = []
    transaction_database['aesthetic'][student_id] = []
    transaction_database['either'][student_id] = []
    
    for index, row in student_df.iterrows():
        is_emotional = row['emotional_reaction'] == 1
        is_aesthetic = row['aesthetically_pleasing'] == 1
        
        if is_emotional:
            transaction_database['emotional'][student_id].append(row['accession_#'])
        if is_aesthetic:
            transaction_database['aesthetic'][student_id].append(row['accession_#'])
        if is_emotional or is_aesthetic:
            transaction_database['either'][student_id].append(row['accession_#'])

In [4]:
def clean_temp(file, df_temp):
    # lowercase and ignore whitespace for consistency
    df_temp.columns = map(str.lower, df_temp.columns)
    new_columns = []
    for column in df_temp.columns:
        new_column = column.strip().replace(" ", "_")
        new_columns.append(new_column)
    df_temp.columns = new_columns

    # correct bad spelling
    if 'asthetically_pleasing' in df_temp.columns:
        df_temp = df_temp.rename(columns={'asthetically_pleasing' : 'aesthetically_pleasing'})

    # abhi's case - student_id missing
    if 'student_id' not in df_temp.columns:
        gen_id = file.split('_')[0]
        df_temp['student_id'] = gen_id
    return df_temp.copy()

In [5]:
for file in directories:
    rel_path = path + file
    if file.endswith('.xlsx'):
        df_temp = pd.read_excel(rel_path)
    
        df_temp = clean_temp(file, df_temp)
        
        # student_id, emotional_reaction, and aesthetically_pleasing
        # are not always there, filter out the ones that don't have one of these columns
        is_valid = 'emotional_reaction' in df_temp.columns and 'aesthetically_pleasing' in df_temp.columns
        
        if(is_valid):
            
            # removed the 'unnamed' feilds
            df_temp = df_temp[valid_cols]
            
            # add an 'either' column
            # df_temp['either'] = np.where((df_temp['emotional_reaction'] | df_temp['aesthetically_pleasing'] > 0), 1, 0)
            df_temp['either'] = df_temp['emotional_reaction'].astype(int) + df_temp['aesthetically_pleasing'].astype(int)
            
            update_transaction_database(df_temp)

            df_master = df_master.append(df_temp, ignore_index = True)

# sort by student_id to make things easier
df_master = df_master.sort_values(by=['student_id'])
df_master
        
#get list of student ids
students = df_master['student_id']
students = students.drop_duplicates()
students

1919    AnaW4804
1702      Frp323
3059    ahasbany
450      akp2597
1322    akshay17
2822    araman18
122     ashk2016
718         ayan
523        benli
2058    brandonn
389     caitlien
1022    cjenwere
1217     colette
1860    ericamtz
1799       gokul
237     gperez13
1505     gskaggs
2997     hh26257
2341    hrithikr
1020         ich
1281     jrm7328
2938    kevliang
41       kjh2858
2222     kjoseph
2040    kushalcd
117       lgm977
2701     manders
2761       maram
1200    mshao123
2433    nithin13
2584      nk9373
2138      nz3222
2555      pa8789
1482         poi
778     pranooha
2339     preston
905     rahulram
1969     raymond
1562       riz74
554     rsmoreno
2673    ryanyz10
2492     serfurt
1423    shaniyur
853     shyampat
1092     simon18
327     snowaski
642      sr46252
257      sra2398
1659     st33578
839     tchatter
2218       viswa
Name: student_id, dtype: object

# Data Analysis
## Questions to Answer:
- What artist was the most likely to elicit an emotional reaction?
- What artist was the most likely to elicit an aesthetically pleasing reaction?
- What pieces of art were the most widely judged to have any reaction?
- What student had the most “1” reactions – find the art lover
- What student had the most “0” reactions – find the art grinch



In [6]:
"""
returns a dictionary tallying the count of a certain attrbute 
for each unique item defined in category

df - dataframe
category - distinct items
tally_by - column to count number of distinct items by
target - target value to add +1 to the tally
"""
def get_count(df, category, tally_by, target, by_one):
    count = {}
    for index, row in df.iterrows():
        
        inc = 1
        if by_one == False:
            inc = row[tally_by]
        
        if row[tally_by] >= target:
            if row[category] not in count:
                count[row[category]] = inc
                
            else:
                count[row[category]] += inc
    return count

In [7]:
def max_count(count):
    return max(count, key=count.get)
def min_count(count):
    return min(count, key=count.get)

In [8]:
# What artist was the most likely to elicit an emotional reaction?
emo_artist = get_count(df_master, 'artist_sort_name', 'emotional_reaction', 1, True)

# What artist was the most likely to elicit an aesthetically pleasing reaction?
aes_artist = get_count(df_master, 'artist_sort_name', 'aesthetically_pleasing', 1, True)

# What pieces of art were the most widely judged to have any reaction?
reaction_art = get_count(df_master, 'title', 'either', 1, True)

# What student had the most “1” reactions – find the art lover
# What student had the most “0” reactions – find the art grinch
lover_student = get_count(df_master, 'student_id', 'either', 1, False)

In [9]:
print("What artist was the most likely to elicit an emotional reaction?")
print(max_count(emo_artist))
print()
print("What artist was the most likely to elicit an aesthetically pleasing reaction?")
print(max_count(aes_artist))
print()
print("What pieces of art were the most widely judged to have any reaction?")
print(max_count(reaction_art))
print()
print("What student had the most “1” reactions – find the art lover")
print(max_count(lover_student))
print()
print("What student had the most “0” reactions – find the art grinch")
print(min_count(lover_student))

What artist was the most likely to elicit an emotional reaction?
Camnitzer, Luis

What artist was the most likely to elicit an aesthetically pleasing reaction?
Anonymous

What pieces of art were the most widely judged to have any reaction?
Untitled

What student had the most “1” reactions – find the art lover
st33578

What student had the most “0” reactions – find the art grinch
ich


## Getting Frequent Itemsets

In [10]:
dataset = []
for student in students:
    art = transaction_database['aesthetic'][student]
    art = [str(artpiece) for artpiece in art]
    dataset.append(art)

In [11]:
te = TransactionEncoder()
te_array = te.fit(dataset).transform(dataset)
df_item = pd.DataFrame(te_array, columns = te.columns_)
df_item = df_item.set_index(students)
df_item.head()

,1977.112,1977.119,1977.25,1977.32,1977.9,1978.46,1978.5,1978.89,1978.93,1979.30,...,P1974.11.2,P1975.18.1/15,P1975.18.10/15,P1975.18.2/15,P1975.18.6/15,P1975.18.7/15,P1975.21.1,P1975.23.3,P1976.10.1,P1976.13.2
student_id,,,,,,,,,,,,,,,,,,,,,
AnaW4804,True,False,False,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
Frp323,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ahasbany,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
akp2597,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
akshay17,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True


In [12]:
#decided to use apriori because trees dont save time here 
#time_ap = time.time()
frequent_itemsets = apriori(df_item, min_support =0.03, use_colnames = True)
#time_ap = time.time() - time_ap

#time_tree = time.time()
#frequent_itemsets_tree = fpgrowth(df_item, min_support =0.03)
#time_tree = time.time() - time_tree
frequent_itemsets

,support,itemsets
0,0.058824,(1977.112)
1,0.098039,(1977.119)
2,0.058824,(1977.25)
3,0.039216,(1977.32)
4,0.078431,(1977.9)
...,...,...
1093,0.039216,"(2005.151, 2017.282, 1986.361.21/25, 2017.1285..."
1094,0.039216,"(2017.1087, 2017.282, 1986.361.21/25, 2017.128..."
1095,0.039216,"(G1976.11.26, 2004.45, 2017.106, 2002.2589, 20..."
1096,0.039216,"(2017.1087, 2005.151, 2017.282, 2017.1285, 201..."


## Generate Association Rules

In [13]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1977.112),(2017.429),0.058824,0.058824,0.039216,0.666667,11.333333,0.035755,2.823529
1,(2017.429),(1977.112),0.058824,0.058824,0.039216,0.666667,11.333333,0.035755,2.823529
2,(1977.119),(1991.187),0.098039,0.156863,0.039216,0.400000,2.550000,0.023837,1.405229
3,(1991.187),(1977.119),0.156863,0.098039,0.039216,0.250000,2.550000,0.023837,1.202614
4,(1977.119),(2002.2171),0.098039,0.039216,0.039216,0.400000,10.200000,0.035371,1.601307


## Apply Association Rules

In [14]:
rules = rules[ (rules['lift'] >= 23) &
      (rules['confidence'] == 1)]

In [15]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
30,(2003.29),(1977.32),0.039216,0.039216,0.039216,1.0,25.5,0.037678,inf
31,(1977.32),(2003.29),0.039216,0.039216,0.039216,1.0,25.5,0.037678,inf
68,(1980.72.10/10),(P1967.1.17/20),0.039216,0.039216,0.039216,1.0,25.5,0.037678,inf
69,(P1967.1.17/20),(1980.72.10/10),0.039216,0.039216,0.039216,1.0,25.5,0.037678,inf
222,(1983.132.21/55),(1995.19),0.039216,0.039216,0.039216,1.0,25.5,0.037678,inf
...,...,...,...,...,...,...,...,...,...
2750,"(2017.282, 2017.912)","(2017.1087, 2017.1285, 2005.151, 1986.361.21/25)",0.039216,0.039216,0.039216,1.0,25.5,0.037678,inf
2751,"(2017.1285, 1986.361.21/25)","(2017.1087, 2005.151, 2017.282, 2017.912)",0.039216,0.039216,0.039216,1.0,25.5,0.037678,inf
2752,"(2017.912, 1986.361.21/25)","(2017.1087, 2017.1285, 2005.151, 2017.282)",0.039216,0.039216,0.039216,1.0,25.5,0.037678,inf
2753,"(2017.1285, 2017.912)","(2017.1087, 2005.151, 2017.282, 1986.361.21/25)",0.039216,0.039216,0.039216,1.0,25.5,0.037678,inf


## Creating Recomendations Per Student

In [16]:
#top three most popular art
temp = get_count(df_master, 'accession_#', 'aesthetically_pleasing', 1, True)
d = coll.Counter(temp)
d.most_common(10)

[('1991.187', 7),
 ('1986.361.21/25', 6),
 ('2017.192.1', 6),
 ('2016.108', 6),
 ('G1976.11.26', 5),
 ('2017.30', 5),
 ('2017.1087', 5),
 ('P1967.1.19/20', 5),
 ('1982.719', 5),
 ('P1973.11.2', 5)]

In [17]:
reqs = {}
for student in students:
    student_req = set()
    for index, data in rules.iterrows():
        stud_likes = transaction_database['aesthetic'][student]
        antc = list(rules.loc[index]['antecedents'])
        for art in stud_likes:
            con = []
            if str(art) in antc:
                con = list(rules.loc[index]['consequents'])
                for conseq in con:
                    student_req.add(conseq)
    if (len(student_req) < 3):
        student_req.update(d.most_common(3 - len(student_req)))
    reqs[student] = list(student_req)[0:3]

In [18]:
reqs

{'AnaW4804': ['1995.259.6/19', '1991.128', '2005.64'],
 'Frp323': ['2017.101.a-b', '2003.29', '2000.37'],
 'ahasbany': ['1992.253.29/35', 'G1974.6', '1977.119'],
 'akp2597': ['2017.1087', 'P1976.13.2', '1999.47'],
 'akshay17': ['2017.1087', 'P1976.13.2', '2000.37'],
 'araman18': ['1993.91', '1999.47', '1991.415'],
 'ashk2016': ['2017.1087', '2017.101.a-b', 'P1976.13.2'],
 'ayan': ['1992.253.29/35', '2017.101.a-b', '1983.124'],
 'benli': ['2018.133', '1977.119', '2016.10'],
 'brandonn': ['2017.1087', 'P1976.13.2', '2000.37'],
 'caitlien': ['2004.82', ('1991.187', 7), ('1986.361.21/25', 6)],
 'cjenwere': ['1994.27', '2003.29', '1977.32'],
 'colette': ['G1969.6.436', '1996.263', '2007.3'],
 'ericamtz': ['2004.45', '2017.473', '2004.114'],
 'gokul': ['2017.1087', '2002.543', '2000.37'],
 'gperez13': ['2017.1087', 'P1976.13.2', '1986.361.21/25'],
 'gskaggs': ['2017.1363', 'G1974.6', '1977.119'],
 'hh26257': [('2017.192.1', 6), ('1991.187', 7), ('1986.361.21/25', 6)],
 'hrithikr': ['2017.410

In [22]:
#make dictionary excell file
df = pd.DataFrame(data=reqs)

df = (df.T)

print (df)

df.to_excel('recomendations.xlsx')

                        0                  1                    2
AnaW4804    1995.259.6/19           1991.128              2005.64
Frp323       2017.101.a-b            2003.29              2000.37
ahasbany   1992.253.29/35            G1974.6             1977.119
akp2597         2017.1087         P1976.13.2              1999.47
akshay17        2017.1087         P1976.13.2              2000.37
araman18          1993.91            1999.47             1991.415
ashk2016        2017.1087       2017.101.a-b           P1976.13.2
ayan       1992.253.29/35       2017.101.a-b             1983.124
benli            2018.133           1977.119              2016.10
brandonn        2017.1087         P1976.13.2              2000.37
caitlien          2004.82      (1991.187, 7)  (1986.361.21/25, 6)
cjenwere          1994.27            2003.29              1977.32
colette       G1969.6.436           1996.263               2007.3
ericamtz          2004.45           2017.473             2004.114
gokul     